# ELMo

In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import time
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0524 06:07:08.096705 140015241787136 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [1]:
pd.set_option('display.max_colwidth', 200)
tf.logging.set_verbosity(tf.logging.WARN)

NameError: name 'pd' is not defined

In [3]:
data_dir = "data"

## Load IMDB Data

In [4]:
imdb_data = pd.read_pickle(f"{data_dir}/imdb_data.pickle.gz")

In [5]:
imdb_data.sample(2)

,data_set,polarity,sentence
7941,test,0,This movie was unbelievably bad... It's gory but the violence is just too much to the point where it looks extremely fake and predictable. Since Everything is shown to you there is nothing left to...
1750,test,0,"Except for the appearance of talented Austin Powers impersonator Richard Halpern, this pic was your run of the mill movie spoof. Dated movie references will not help audiences that may come across..."


## Cleaning Text

In [6]:
max_words = 128

text_http_re  = re.compile(r'http\S+')
text_digit_re = re.compile(r'[0-9]')
text_html_re  = re.compile(r'<[^>]{0,20}>')
text_punc_re  = re.compile('[' + re.escape('\'!"#$%&()*+-/:;<=>?@[\\]^_`{|}~') + ']')
text_ws_re    = re.compile('\s+')

nlp = spacy.load('en', disable=['parser', 'ner'])

def clean_text(text):
    text = text.lower()
    text = text_http_re.sub('', text)
    text = text_html_re.sub('', text)
    text = text_digit_re.sub(' ', text)
    text = text_punc_re.sub('', text)
    text = text_ws_re.sub(' ', text)
    text = text.strip()
    return text

def lemmatize(text):
    return ' '.join([token.lemma_ for token in nlp(text)][0:max_words])

def process_text(text):
    return lemmatize(clean_text(text))

In [7]:
imdb_data['clean_review'] = imdb_data.sentence.apply(process_text)

## Extract ELMo Embeddings

In [8]:
# TODO: bryan make it classy      
def elmo_create_embedding_extractor(module, chunk_size=20):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module, trainable=True)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
        
    def __call__(x):
        return session.run(embeddings, { sentences: x })
        
    def batch_extract(xs):
        results = []
        chunk = []
        for x in xs:
            chunk.append(x)
            if len(chunk) >= chunk_size:
                results.extend(extract(chunk))
                print(f"[{datetime.now()}] Extracted {len(results)}")
                chunk = []
        if len(chunk) > 0:
            results.extend(extract(chunk))
            print(f"[{datetime.now()}] Extracted {len(results)}")
        return results

    return batch_extract

elmo_get_embedding = ElmoExtractor("https://tfhub.dev/google/elmo/2", 250)
elmo_get_embedding.__call__()

elmo_get_embedding = elmo_create_embedding_extractor("https://tfhub.dev/google/elmo/2", 250)

Instructions for updating:
Colocations handled automatically by placer.


W0524 06:16:32.179328 140015241787136 deprecation.py:323] From /anaconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0524 06:16:32.924695 140015241787136 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


I0524 06:16:33.155911 140015241787136 monitored_session.py:222] Graph was finalized.


INFO:tensorflow:Running local_init_op.


I0524 06:16:33.530931 140015241787136 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0524 06:16:33.583106 140015241787136 session_manager.py:493] Done running local_init_op.


In [ ]:
current_time = datetime.now()
imdb_data['embedding'] = elmo_get_embedding(imdb_data.clean_review.values)
print("Extraction took time ", datetime.now() - current_time)

[2019-05-24 06:17:44.920189] Extracted 250
[2019-05-24 06:18:47.107878] Extracted 500
[2019-05-24 06:19:49.436713] Extracted 750
[2019-05-24 06:20:51.859553] Extracted 1000
[2019-05-24 06:21:54.266682] Extracted 1250
[2019-05-24 06:22:57.038668] Extracted 1500
[2019-05-24 06:23:59.160260] Extracted 1750
[2019-05-24 06:25:01.422989] Extracted 2000
[2019-05-24 06:26:05.055127] Extracted 2250
[2019-05-24 06:27:08.458267] Extracted 2500
[2019-05-24 06:28:11.884463] Extracted 2750
[2019-05-24 06:29:16.408902] Extracted 3000
[2019-05-24 06:30:19.219166] Extracted 3250
[2019-05-24 06:31:21.403094] Extracted 3500
[2019-05-24 06:32:23.584571] Extracted 3750
[2019-05-24 07:02:12.381073] Extracted 11000
[2019-05-24 07:03:13.605923] Extracted 11250
[2019-05-24 07:04:14.727710] Extracted 11500
[2019-05-24 07:05:15.887587] Extracted 11750
[2019-05-24 07:06:17.024850] Extracted 12000
[2019-05-24 07:07:18.022386] Extracted 12250


In [ ]:
imdb_data.to_pickle(f"{data_dir}/imdb_data_w_elmo_embedding.pickle.gz")